# Load files

In [2]:
import pandas as pd
import glob

col_names = ["Flow ID","Source IP","Source Port","Destination IP","Destination Port","Protocol","Timestamp","Flow Duration","Total Fwd Packets","Total Backward Packets","Total Length of Fwd Packets","Total Length of Bwd Packets","Fwd Packet Length Max","Fwd Packet Length Min","Fwd Packet Length Mean","Fwd Packet Length Std","Bwd Packet Length Max","Bwd Packet Length Min","Bwd Packet Length Mean","Bwd Packet Length Std","Flow Bytes/s","Flow Packets/s","Flow IAT Mean","Flow IAT Std","Flow IAT Max","Flow IAT Min","Fwd IAT Total","Fwd IAT Mean","Fwd IAT Std","Fwd IAT Max","Fwd IAT Min","Bwd IAT Total","Bwd IAT Mean","Bwd IAT Std","Bwd IAT Max","Bwd IAT Min","Fwd PSH Flags","Bwd PSH Flags","Fwd URG Flags","Bwd URG Flags","Fwd Header Length","Bwd Header Length","Fwd Packets/s","Bwd Packets/s","Min Packet Length","Max Packet Length","Packet Length Mean","Packet Length Std","Packet Length Variance","FIN Flag Count","SYN Flag Count","RST Flag Count","PSH Flag Count","ACK Flag Count","URG Flag Count","CWE Flag Count","ECE Flag Count","Down/Up Ratio","Average Packet Size","Avg Fwd Segment Size","Avg Bwd Segment Size","Fwd Header Length","Fwd Avg Bytes/Bulk","Fwd Avg Packets/Bulk","Fwd Avg Bulk Rate","Bwd Avg Bytes/Bulk","Bwd Avg Packets/Bulk","Bwd Avg Bulk Rate","Subflow Fwd Packets","Subflow Fwd Bytes","Subflow Bwd Packets","Subflow Bwd Bytes","Init_Win_bytes_forward","Init_Win_bytes_backward","act_data_pkt_fwd","min_seg_size_forward","Active Mean","Active Std","Active Max","Active Min","Idle Mean","Idle Std","Idle Max","Idle Min","label"]

path = '/Users/xihajun/Documents/GSoC/OWASP-Seraphimdroid/data/Benign/Benign2017/' # use your path
allFiles = glob.glob(path + "/*.csv")
frame = pd.DataFrame()
list_ = []
for file_ in allFiles:
    df = pd.read_csv(file_,index_col=None, header=0, names = col_names)
    list_.append(df)

path = '/Users/xihajun/Documents/GSoC/OWASP-Seraphimdroid/data/Adware/'
allFiles = glob.glob(path + "*/*.csv")
for file_ in allFiles:
    df = pd.read_csv(file_,index_col=None, header=0, names = col_names)
    list_.append(df)
    
frame = pd.concat(list_)

frame['label'].unique()

frame['label'] = frame['label'].replace(['ADWARE_SELFMITE', 'ADWARE_KOODOUS', 'ADWARE_KEMOGE',
       'ADWARE_GOOLIGAN', 'ADWARE_MOBIDASH', 'ADWARE_YOUMI',
       'ADWARE_DOWGIN', 'ADWARE_EWIND', 'ADWARE_FEIWO', 'ADWARE_SHUANET'], 'malware')

/Users/xihajun/anaconda3/lib/python3.7/site-packages/pandas/io/parsers.py:678: UserWarning: Duplicate names specified. This will raise an error in the future.
  return _read(filepath_or_buffer, kwds)


In [3]:
def categorical_converter(data):
    '''manually encode the three categorical columns into ordinals'''
    cols=["Flow ID","Source IP","Destination IP","Destination Port","Protocol","Timestamp",'label']
    # 1 is benign, 0 is malware

    for i in cols:
        data[i] = le.fit_transform(data[i])
    data = hot.fit_transform(data.loc[:,cols])
    
raw_data = frame

In [4]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
le = LabelEncoder()
hot = OneHotEncoder(sparse=True, handle_unknown='ignore')
categorical_converter(raw_data)

In [11]:
features= raw_data.iloc[:,:raw_data.shape[1]]
labels= raw_data.iloc[:,raw_data.shape[1]-1:]
labels= labels.values.ravel() # this becomes a 'horizontal' array

from sklearn.model_selection import train_test_split
df= pd.DataFrame(features)
# slipt training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df, labels, train_size=0.8, test_size=0.2)

X_train_features= X_train.iloc[:,:raw_data.shape[1]-1]

# normalization

In [12]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler().fit(X_train_features)
#normalization of training dataset 
X_train.iloc[:,:raw_data.shape[1]-1] = scaler.transform(X_train.iloc[:,:raw_data.shape[1]-1])

/Users/xihajun/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Users/xihajun/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  """
/Users/xihajun/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


# Kmeans

In [7]:
#labels
X_train_transformed = pd.DataFrame(X_train)
X = X_train_transformed[X_train_transformed.label==1]
X1 = X.iloc[:,:84]
X = X_train_transformed[X_train_transformed.label==0]
X0 = X.iloc[:,:84]

from sklearn.cluster import KMeans

numberofclasses = 20
kmeans1 = KMeans(n_clusters=numberofclasses, random_state=0).fit(X1)
kmeans0 = KMeans(n_clusters=numberofclasses, random_state=0).fit(X0)

# 1NN

In [8]:
import numpy as np
label_train = np.append(np.zeros(numberofclasses)+1,np.zeros(numberofclasses))
data_train = np.append(kmeans1.cluster_centers_,kmeans0.cluster_centers_,axis=0)

from sklearn.neighbors import KNeighborsClassifier 

knn = KNeighborsClassifier(n_neighbors=1,weights = 'distance')
knn.fit(data_train, label_train)

score=knn.score(X_train.iloc[:,:raw_data.shape[1]-1],X_train.iloc[:,raw_data.shape[1]-1:],sample_weight=None)
# inter acc
score

0.8343130280679422

In [9]:
X_test.iloc[:,:raw_data.shape[1]-1] = scaler.transform(X_test.iloc[:,:raw_data.shape[1]-1])

score=knn.score(X_test.iloc[:,:raw_data.shape[1]-1],X_test.iloc[:,raw_data.shape[1]-1:],sample_weight=None)
# testing acc
score

/Users/xihajun/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  """Entry point for launching an IPython kernel.
/Users/xihajun/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


0.8328850893887866

In [10]:
predicted = knn.predict(X_test.iloc[:,:raw_data.shape[1]-1])
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(X_test.iloc[:,raw_data.shape[1]-1:], predicted)
print(confusion_matrix)
from sklearn.metrics import classification_report
report = classification_report(X_test.iloc[:,raw_data.shape[1]-1:], predicted, digits=4)
print(report)

[[69892 12017]
 [14764 63582]]
              precision    recall  f1-score   support

           0     0.8256    0.8533    0.8392     81909
           1     0.8410    0.8116    0.8260     78346

   micro avg     0.8329    0.8329    0.8329    160255
   macro avg     0.8333    0.8324    0.8326    160255
weighted avg     0.8331    0.8329    0.8328    160255



In [ ]:
# try 1nn without kmeans

In [ ]:
from sklearn.neighbors import KNeighborsClassifier 

knn = KNeighborsClassifier(n_neighbors=1,weights = 'distance')
knn.fit(X_train, y_train)
score=knn.score(X_train.iloc[:,:raw_data.shape[1]-1],X_train.iloc[:,raw_data.shape[1]-1:],sample_weight=None)

